In [0]:
import base64
import hashlib
import sys


KEY = b'akKJ6779n9*N76*65764876Ngyusdgfh86%(t98'
b64_chars = [c for c in 'ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz0123456789+/=']

def convert(string, type):
	hash = hashlib.sha512(KEY).hexdigest()
	cipher = b64_chars[:]
	for c in hash:
		char_int = int(c, 16)
		pos = 65 * (char_int / 15)

		cipher.insert(0, cipher.pop(int(pos)-1))
		cipher = cipher[::-1]

	sbox = {}
	for i, c in enumerate(b64_chars):
		sbox[c] = cipher[i]
	if type == 'd':
		   sbox = dict((v, k) for k, v in sbox.items())

	for i, c in enumerate(string):
		string[i] = sbox[c]

	return ''.join(string)

def encrypt(string):
	# base64 encode plaintext and convert to list of characters
	string = [c for c in base64.b64encode(string.encode()).decode()]

	return convert(string, 'e')

def decrypt(string):
	string = [c for c in string.strip()]

	return base64.b64decode(convert(string, 'd')).decode()



In [0]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from collections import Counter

%matplotlib inline
import matplotlib.pyplot as plt

train = pd.read_csv('training.csv', index_col='index')


In [0]:
X_=(train["text"].values)
X=[]
for text in X_:
   X.append(text)


In [0]:
Y=[]
for text in X:
  Y.append(encrypt(text))
  

In [0]:
word2id={}
idx=0
for text in X:
  for c in text:
    if  not  c in word2id.keys() :
      word2id[c]=idx
      idx+=1
id2word = {v: k for k, v in word2id.items()}

In [6]:
len(id2word.keys())

78

In [0]:
v=len(id2word.keys())


In [0]:
word2id_c={}
idx=0
for text in Y:
  for c in text:
    if  not  c in word2id_c.keys() :
      word2id_c[c]=idx
      idx+=1
id2word_c = {v: k for k, v in word2id_c.items()}

In [0]:
n=len(word2id_c.keys())

In [0]:
plain_text=X
ciphertext=Y

In [0]:
X_=X
X=[]

In [0]:
for text in X_:
  tmp=[]
  for c in text:
    tmp.append(word2id[c])
  X.append(tmp)


In [0]:
Y_=Y
Y=[]
for text in Y_:
  tmp=[]
  for c in text:
    tmp.append(word2id_c[c])
  Y.append(tmp)


In [0]:
import numpy as np
A=np.zeros((v,v))

p=np.zeros((v,n))

begin=np.zeros(v)
end=np.zeros(v)



In [0]:
def Markov_Prob(D,i,j):
   total=0
   s=0
   for d in D:
     for k in range(1,len(d)):
       if d[k-1]==i :
         total+=1
       if d[k]==j and d[k-1]==i:
           s+=1
   
   A[i,j]=s/total




In [0]:
for i in range(v):
  for j in range(v):       
       Markov_Prob(X,i,j)

In [0]:
for i in range(v):
  for d in X:
    if d[0]==i :
       begin[i]+=1
begin/=sum(begin)


In [0]:

for i in range(v):
  for d in X:
    if d[-1]==i :
       end[i]+=1
end/=sum(end)


In [0]:
def cipher_Prob(X,Y,i,j):
   for d in range(len(X)):
     O=X[d]
     C=Y[d] 
     for k in range(len(O)):
       if O[k]==j and C[k]==i :
           p[j,i]+=1
     
for i in range(n):
  for j in range(v):
       cipher_Prob(X,Y,i,j)

for i in range(n):
  p[i,:]/=sum(p[i,:])

In [0]:
s=np.ones((v,n))

In [0]:
for i in range(v):
  s[i,:]/=sum(s[i,:])


In [0]:
for epoch in range(1):
  print(epoch)
  count=np.zeros((v,n))
  Q=np.zeros((v,n))

  for i in range(v):
    Q[i,0]=begin[i]

  for j in range(1,n):
   for i in range(v):
     for k in range(v):         
          Q[i,j]+=np.exp(np.log(Q[k,j-1])+np.log(A[i,k])+np.exp(s[k,j-1]))

  

  R=np.ones((v,n))
  for i in range(v):
    R[i,-1]=end[i]
  for j in range(n-2,-1,-1):
    for i in range(v):
      R[i,j]=0
      for k in range(v):
        R[i,j]+=np.exp( np.log(R[k,j+1])+np.log(A[i,k])+np.log(s[k,j+1])  )
  
  for j in range(n):
    for i in range(v):
     if not p[i,j]==0:
      count[i,j]+=np.exp(np.log(Q[i,j])+np.log(R[i,j])+np.log(p[i,j]))
  for i in range(v):
    if not sum(count[i,:])==0 :
     count[i,:]/=sum(count[i,:])
  s=count

  

In [0]:
def prediction(text):
  out=[]
  c=text[0]

  prod=begin[:]*s[:,c]
  out.append(np.argmax(prod))
  
  prod=np.exp(np.log(begin[out[-1]])+np.log(s[out[-1],c]))

  for i in range(1,len(text-1)):
    c=text[i]
    tmp=prod
    prod=prod*A[out[-1],:]*s[out[-1],:]

    out.append(np.argmax(prod))

    prod=np.exp( np.log(tmp) + np.log(A[out[-1],:])+np.log(s[out[-1],:]))
  
  c=text[-1]

  tmp=prod

  prod=prod*end[out[-1],:]*s[out[-1],:]
  out.append(np.argmax(prod))

  return out 



